# Creating Geopackage and upload to Geoserver as layer
- Created: 7/1/2016

In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import os, geopandas as gpd, pandas as pd
import fiona as fio, json, numpy as np
from shapely.geometry import Point
from geopandas import GeoDataFrame

%load_ext version_information
%version_information numpy, pandas, geopandas, fiona, shapely, gdal

/home/lsetiawan/anaconda2/envs/geo/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Software versions
Python 2.7.12 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 4.2.0 36 generic x86_64 with debian jessie sid
numpy 1.11.1
pandas 0.18.1
geopandas 0+unknown
fiona 1.6.4
shapely 1.5.16
gdal 1.11.4
Fri Jul 01 15:37:44 2016 PDT

In [2]:
# Assign file_path
pth = "./"
fcsv = "platform_meta_201606291445_1.csv"

In [3]:
def readCSV(pth,fcsv):
    df = pd.read_csv(os.path.join(pth,fcsv))
    df_siso = df[df['platform_class'] == 'SISO']
    df_siso_all = df_siso[df_siso['mode'] == 'all']
    df_cleaned = df_siso_all.drop(['mode','app_id','bounding_area','platform_class','route','route_hue','has_obs_data'], axis=1)
    
    # Convert data to str?
    '''for d in df_cleaned:
        #pass
        if d == 'lat' or d == 'lon' or d == 'geometry':
            pass
        else:
            #pass
            df_cleaned['{}'.format(d)] = df_cleaned['{}'.format(d)].astype(str)'''

    df_final = df_cleaned.reset_index(drop=True)
    
    return df_final

In [4]:
data = readCSV(pth,fcsv)
# Look at their types
for d in data:
    print(type(data['{}'.format(d)][0]))

<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'numpy.float64'>
<type 'numpy.float64'>
<type 'numpy.int64'>
<type 'str'>
<type 'numpy.float64'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'numpy.int64'>


In [5]:
data.head()

,platform_label,platform_type,name,short_name,datasrc_platform_cd,data_timezone,lat,lon,bounding_area_hue,state,...,region,provider,provider_url,provider_type,data_source,data_source_url,url,icon,status,status_date
0,APL_Chaba,Buoy,?há?ba· UW/NANOOS Moored Buoy near La Push,APL-UW ?há?ba·,aplchaba,US/Pacific,47.9672,-124.9502,0,Washington,...,La Push,APL-UW,http://www.nanoos.org/,Academic,NANOOS-APL2,http://www.nanoos.org/,http://wavechasers.apl.washington.edu/projects...,buoy_default,online,1458680400
1,APL_Nemo,Buoy,UW/NANOOS NEMO Subsurface profiler with NOAA E...,APL-UW NEMO-ESP Profiler,aplnemo,US/Pacific,47.9740,-124.9550,0,Washington,...,La Push,APL-UW,http://www.nanoos.org,Academic,NANOOS-APL3,http://www.nanoos.org,NaN,buoy_default,online,1464030000
2,APL_NPB1PtWells,Buoy,Profiling Buoy at Point Wells - Central Sound,ORCA Point Wells,npb1,US/Pacific,47.7612,-122.3972,0,Washington,...,Puget Sound,ORCA-UW,http://orca.ocean.washington.edu,Academic,NANOOS-APL,NaN,http://orca.ocean.washington.edu/data_pointWel...,buoy_default,online,1454396400
3,APL_NPB2Carr,Buoy,Profiling Buoy at Carr Inlet - South Sound,ORCA Carr Inlet,npb2,US/Pacific,47.2800,-122.7300,0,Washington,...,Puget Sound,ORCA-UW,http://orca.ocean.washington.edu,Academic,NANOOS-APL,NaN,http://orca.ocean.washington.edu/data_carrInle...,buoy_default,online,1430422200
4,CDIP_46211,Buoy,Station 46211 - Grays Harbor (036),CDIP Grays Harbor,46211,US/Pacific,46.8580,-124.2440,0,Washington,...,Grays Harbor,CDIP-Scripps,http://cdip.ucsd.edu/,Academic,NDBC,NaN,http://cdip.ucsd.edu/themes/s?u2=s:036:st:1&d2=p9,buoy_default,online,1458676800


In [6]:
def createGeoJSON(data):
    # Assign CRS, retrieved from epsg.io, the example below is EPSG:4326
    crs = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'
    geometry = [Point(xy) for xy in zip(data.lon, data.lat)]
    geo_df = GeoDataFrame(data, crs=crs, geometry=geometry).apply(pd.to_numeric, errors='ignore')
    
    #json_data = geo_df.to_json()
    
    print("Original Column Header: {}\n".format(geo_df.columns.values))
    
    return geo_df, crs

In [7]:
gdf,crs = createGeoJSON(data)

Original Column Header: ['platform_label' 'platform_type' 'name' 'short_name' 'datasrc_platform_cd'
 'data_timezone' 'lat' 'lon' 'bounding_area_hue' 'state' 'sub_region'
 'region' 'provider' 'provider_url' 'provider_type' 'data_source'
 'data_source_url' 'url' 'icon' 'status' 'status_date' 'geometry']



In [8]:
gdf.head()

,platform_label,platform_type,name,short_name,datasrc_platform_cd,data_timezone,lat,lon,bounding_area_hue,state,...,provider,provider_url,provider_type,data_source,data_source_url,url,icon,status,status_date,geometry
0,APL_Chaba,Buoy,?há?ba· UW/NANOOS Moored Buoy near La Push,APL-UW ?há?ba·,aplchaba,US/Pacific,47.9672,-124.9502,0,Washington,...,APL-UW,http://www.nanoos.org/,Academic,NANOOS-APL2,http://www.nanoos.org/,http://wavechasers.apl.washington.edu/projects...,buoy_default,online,1458680400,POINT (-124.9502 47.9672)
1,APL_Nemo,Buoy,UW/NANOOS NEMO Subsurface profiler with NOAA E...,APL-UW NEMO-ESP Profiler,aplnemo,US/Pacific,47.9740,-124.9550,0,Washington,...,APL-UW,http://www.nanoos.org,Academic,NANOOS-APL3,http://www.nanoos.org,NaN,buoy_default,online,1464030000,POINT (-124.955 47.974)
2,APL_NPB1PtWells,Buoy,Profiling Buoy at Point Wells - Central Sound,ORCA Point Wells,npb1,US/Pacific,47.7612,-122.3972,0,Washington,...,ORCA-UW,http://orca.ocean.washington.edu,Academic,NANOOS-APL,NaN,http://orca.ocean.washington.edu/data_pointWel...,buoy_default,online,1454396400,POINT (-122.3972 47.7612)
3,APL_NPB2Carr,Buoy,Profiling Buoy at Carr Inlet - South Sound,ORCA Carr Inlet,npb2,US/Pacific,47.2800,-122.7300,0,Washington,...,ORCA-UW,http://orca.ocean.washington.edu,Academic,NANOOS-APL,NaN,http://orca.ocean.washington.edu/data_carrInle...,buoy_default,online,1430422200,POINT (-122.73 47.28)
4,CDIP_46211,Buoy,Station 46211 - Grays Harbor (036),CDIP Grays Harbor,46211,US/Pacific,46.8580,-124.2440,0,Washington,...,CDIP-Scripps,http://cdip.ucsd.edu/,Academic,NDBC,NaN,http://cdip.ucsd.edu/themes/s?u2=s:036:st:1&d2=p9,buoy_default,online,1458676800,POINT (-124.244 46.858)


In [9]:
fio.supported_drivers

{'ARCGEN': 'r',
 'AeronavFAA': 'r',
 'BNA': 'raw',
 'DGN': 'raw',
 'DXF': 'raw',
 'ESRI Shapefile': 'raw',
 'FileGDB': 'raw',
 'GMT': 'raw',
 'GPKG': 'rw',
 'GPSTrackMaker': 'raw',
 'GPX': 'raw',
 'GeoJSON': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'OpenFileGDB': 'r',
 'PCIDSK': 'r',
 'PDS': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

In [10]:
def common_geom_type(df):
    # Need to check geom_types before we write to file...
    # Some (most?) providers expect a single geometry type:
    # Point, LineString, or Polygon
    geom_types = df.geometry.geom_type.unique()

    from os.path import commonprefix   # To find longest common prefix
    geom_type = commonprefix([g[::-1] for g in geom_types if g])[::-1]  # Reverse
    if not geom_type:
        geom_type = None

    return geom_type

In [11]:
def infer_schema(df):
    try:
        from collections import OrderedDict
    except ImportError:
        from ordereddict import OrderedDict

    def convert_type(in_type):
        if in_type == object:
            return 'str'
        out_type = type(np.asscalar(np.zeros(1, in_type))).__name__
        if out_type == 'long':
            out_type = 'int'
        return out_type

    properties = OrderedDict([
        (col, convert_type(_type)) for col, _type in
        zip(df.columns, df.dtypes) if col != df._geometry_column_name
    ])

    geom_type = common_geom_type(df)
    if not geom_type:
        raise ValueError("Geometry column cannot contain mutiple "
                         "geometry types when writing to file.")

    schema = {'geometry': geom_type, 'properties': properties}

    return schema

In [12]:
schema = infer_schema(gdf)
schema

{'geometry': 'Point',
 'properties': OrderedDict([('platform_label', 'str'),
              ('platform_type', 'str'),
              ('name', 'str'),
              ('short_name', 'str'),
              ('datasrc_platform_cd', 'str'),
              ('data_timezone', 'str'),
              ('lat', 'float'),
              ('lon', 'float'),
              ('bounding_area_hue', 'int'),
              ('state', 'str'),
              ('sub_region', 'float'),
              ('region', 'str'),
              ('provider', 'str'),
              ('provider_url', 'str'),
              ('provider_type', 'str'),
              ('data_source', 'str'),
              ('data_source_url', 'str'),
              ('url', 'str'),
              ('icon', 'str'),
              ('status', 'str'),
              ('status_date', 'int')])}

In [13]:
features = [g for g in gdf.iterfeatures()]
features[0]

{'geometry': {'coordinates': (-124.9502, 47.9672), 'type': 'Point'},
 'id': '0',
 'properties': {'bounding_area_hue': 0,
  'data_source': 'NANOOS-APL2',
  'data_source_url': 'http://www.nanoos.org/',
  'data_timezone': 'US/Pacific',
  'datasrc_platform_cd': 'aplchaba',
  'icon': 'buoy_default',
  'lat': 47.9672,
  'lon': -124.9502,
  'name': '?h\xc3\xa1?ba\xc2\xb7 UW/NANOOS Moored Buoy near La Push',
  'platform_label': 'APL_Chaba',
  'platform_type': 'Buoy',
  'provider': 'APL-UW',
  'provider_type': 'Academic',
  'provider_url': 'http://www.nanoos.org/',
  'region': 'La Push',
  'short_name': 'APL-UW ?h\xc3\xa1?ba\xc2\xb7',
  'state': 'Washington',
  'status': 'online',
  'status_date': 1458680400,
  'sub_region': None,
  'url': 'http://wavechasers.apl.washington.edu/projects/active-projects/nemo'},
 'type': 'Feature'}

In [14]:
def createGPKG(features,schema,crs):
    
    #out_pth = '/home/geoserver2_7/gs_data_dir/data/geopackages/'
    out_pth = '/home/lsetiawan/Desktop'
    
    with fio.open(os.path.join(out_pth,'nanoos_nvs.gpkg'), 'w', driver="GPKG", schema = schema,crs_wkt=crs) as c:
        c.writerecords(features)

In [15]:
#if os.path.exists('/home/lsetiawan/Desktop/nanoos_nvs.gpkg'):
#    print('Deleted')
#    os.remove('/home/lsetiawan/Desktop/nanoos_nvs.gpkg')

print('Created')
createGPKG(features,schema,crs)

Created


TypeError: function takes exactly 5 arguments (1 given)